## Deliverable 2. Create a Customer Travel Destinations Map.

In [36]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [37]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thompson,CA,55.7435,-97.8558,32.00,40,40,2.30,scattered clouds
1,1,Mar Del Plata,AR,-38.0023,-57.5575,75.99,83,40,12.66,scattered clouds
2,2,Camargo,MX,27.6667,-105.1667,72.66,37,0,2.15,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,48.20,53,20,2.30,few clouds
4,4,Boende,CD,-0.2167,20.8667,75.45,85,96,2.42,overcast clouds


In [38]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [39]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Mar Del Plata,AR,-38.0023,-57.5575,75.99,83,40,12.66,scattered clouds
4,4,Boende,CD,-0.2167,20.8667,75.45,85,96,2.42,overcast clouds
6,6,Salalah,OM,17.0151,54.0924,82.40,78,0,3.44,clear sky
7,7,Mahebourg,MU,-20.4081,57.7000,81.00,75,12,7.00,few clouds
14,14,Carnarvon,AU,-24.8667,113.6333,77.00,78,34,17.27,scattered clouds
16,16,Sambava,MG,-14.2667,50.1667,79.88,83,10,6.42,clear sky
22,22,Mecca,SA,21.4267,39.8261,80.73,31,81,8.81,broken clouds
23,23,Atuona,PF,-9.8000,-139.0333,80.29,80,66,10.89,broken clouds
24,24,Asyut,EG,27.1810,31.1837,78.80,18,0,16.11,clear sky
26,26,Karratha,AU,-20.7377,116.8463,83.50,47,2,6.73,clear sky


In [40]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                232
City                   232
Country                229
Lat                    232
Lng                    232
Max Temp               232
Humidity               232
Cloudiness             232
Wind Speed             232
Current Description    232
dtype: int64

In [41]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [42]:
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [43]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mar Del Plata,AR,75.99,scattered clouds,-38.0023,-57.5575,
4,Boende,CD,75.45,overcast clouds,-0.2167,20.8667,
6,Salalah,OM,82.40,clear sky,17.0151,54.0924,
7,Mahebourg,MU,81.00,few clouds,-20.4081,57.7000,
14,Carnarvon,AU,77.00,scattered clouds,-24.8667,113.6333,
16,Sambava,MG,79.88,clear sky,-14.2667,50.1667,
22,Mecca,SA,80.73,broken clouds,21.4267,39.8261,
23,Atuona,PF,80.29,broken clouds,-9.8000,-139.0333,
24,Asyut,EG,78.80,clear sky,27.1810,31.1837,
26,Karratha,AU,83.50,clear sky,-20.7377,116.8463,


In [44]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [45]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mar Del Plata,AR,75.99,scattered clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
4,Boende,CD,75.45,overcast clouds,-0.2167,20.8667,
6,Salalah,OM,82.40,clear sky,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
7,Mahebourg,MU,81.00,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
14,Carnarvon,AU,77.00,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
16,Sambava,MG,79.88,clear sky,-14.2667,50.1667,Melrose
22,Mecca,SA,80.73,broken clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
23,Atuona,PF,80.29,broken clouds,-9.8000,-139.0333,Villa Enata
24,Asyut,EG,78.80,clear sky,27.1810,31.1837,Florence Hotel Assiut
26,Karratha,AU,83.50,clear sky,-20.7377,116.8463,ibis Styles Karratha


In [46]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mar Del Plata,AR,75.99,scattered clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
4,Boende,CD,75.45,overcast clouds,-0.2167,20.8667,
6,Salalah,OM,82.40,clear sky,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
7,Mahebourg,MU,81.00,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
14,Carnarvon,AU,77.00,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
16,Sambava,MG,79.88,clear sky,-14.2667,50.1667,Melrose
22,Mecca,SA,80.73,broken clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
23,Atuona,PF,80.29,broken clouds,-9.8000,-139.0333,Villa Enata
24,Asyut,EG,78.80,clear sky,27.1810,31.1837,Florence Hotel Assiut
26,Karratha,AU,83.50,clear sky,-20.7377,116.8463,ibis Styles Karratha


In [47]:
clean_hotel_df = clean_hotel_df[clean_hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mar Del Plata,AR,75.99,scattered clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
6,Salalah,OM,82.40,clear sky,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
7,Mahebourg,MU,81.00,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
14,Carnarvon,AU,77.00,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
16,Sambava,MG,79.88,clear sky,-14.2667,50.1667,Melrose
22,Mecca,SA,80.73,broken clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
23,Atuona,PF,80.29,broken clouds,-9.8000,-139.0333,Villa Enata
24,Asyut,EG,78.80,clear sky,27.1810,31.1837,Florence Hotel Assiut
26,Karratha,AU,83.50,clear sky,-20.7377,116.8463,ibis Styles Karratha
28,Nhulunbuy,AU,80.60,broken clouds,-12.2333,136.7667,East Arnhem Real Estate


In [51]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [53]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))